In [1]:
!pip install ultralytics opencv-python


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\91812\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Run it Before running any of the Below Code

In [ ]:
# This is comment This is Basic First Code for Initializing Varibles 
#second line comment
import os
import cv2
import numpy as np
from ultralytics import YOLO

# --- USER: VERIFY YOUR PROJECT DIRECTORY ---
PROJECT_DIR = "/content/drive/MyDrive/Colab_Projects/AI-Based-Intelligent-Traffic-Monitoring-and-Management-System"
# The name of your input video file, which should be in the 'data' subfolder
INPUT_VIDEO_FILENAME = "input.mp4"
# ---

# --- USER: DEFINE YOUR LANE'S ROI HERE ---
# Define the coordinates of YOUR lane's polygon
ROI_POINTS = np.array([(100, 300), (800, 300), (850, 450), (50, 450)], np.int32)
# ---

# Construct full paths for video files
input_video_path = os.path.join(PROJECT_DIR, 'data', INPUT_VIDEO_FILENAME)
output_video_path = os.path.join(PROJECT_DIR, 'output', 'output_vehicle_count.mp4')

# Ensure the output directory exists
os.makedirs(os.path.join(PROJECT_DIR, 'output'), exist_ok=True)

# Change the current working directory to the project folder
os.chdir(PROJECT_DIR)

# Load the pre-trained YOLOv8 model
print("Loading YOLOv8 model...")
model = YOLO('yolov8n.pt')

# COCO class IDs for vehicles (car, motorcycle, bus, truck)
VEHICLE_CLASS_IDS = [2, 3, 5, 7]

Loading YOLOv8 model...


# Video Processing especially for google Colab

In [ ]:
# Open the input video file
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print(f"Error: Could not open video at {input_video_path}")
else:
    # Get video properties for the output file
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    print(f"Processing video: {INPUT_VIDEO_FILENAME}...")

try:
    while True:
        success, frame = cap.read()
        if not success:
            break # End of video

        # This check prevents errors if a frame is corrupted
        if frame.shape[1] != frame_width or frame.shape[0] != frame_height:
            frame = cv2.resize(frame, (frame_width, frame_height))

        # Reset counter for each frame
        vehicle_count = 0

        # Draw the ROI polygon once per frame
        cv2.polylines(frame, [ROI_POINTS], isClosed=True, color=(255, 0, 0), thickness=2)

        # Run YOLO detection
        results = model(frame)

        # Process detections
        for box in results[0].boxes:
            if int(box.cls) in VEHICLE_CLASS_IDS:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

                if cv2.pointPolygonTest(ROI_POINTS, (center_x, center_y), False) >= 0:
                    vehicle_count += 1
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Write the final count and the frame to the output video
        cv2.putText(frame, f"Vehicles in Lane: {vehicle_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        out.write(frame)

finally:
    # This ensures files are closed properly even if an error occurs
    print("Processing complete. Releasing resources.")
    cap.release()
    out.release()

🚀 Starting live detection... Press 'q' to quit.

0: 480x640 4 persons, 271.7ms
Speed: 13.7ms preprocess, 271.7ms inference, 21.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 424.8ms
Speed: 13.4ms preprocess, 424.8ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 198.6ms
Speed: 10.3ms preprocess, 198.6ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 243.6ms
Speed: 7.5ms preprocess, 243.6ms inference, 3.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 201.2ms
Speed: 1.8ms preprocess, 201.2ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 213.1ms
Speed: 1.9ms preprocess, 213.1ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 persons, 152.0ms
Speed: 3.9ms preprocess, 152.0ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 171.7ms
Speed: 2.4ms p

# Accessing Through Web Cam on Local Pc "Live"

In [ ]:


# --- Load YOLO model ---
model = YOLO("yolov8n.pt")

# --- ROI (adjust to your camera view) ---
ROI_POINTS = np.array([(150, 100), (600, 100), (600, 400), (150, 400)], np.int32)

# Vehicle class IDs (car=2, motorcycle=3, bus=5, truck=7)
VEHICLE_CLASS_IDS = [2, 3, 5, 7]

# --- Open webcam ---
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # try without CAP_DSHOW if on Linux/Mac

if not cap.isOpened():
    print("❌ Error: Could not open webcam.")
    exit()

print("🚀 Starting live detection... Press 'q' to quit.")

try:
    while True:
        success, frame = cap.read()
        if not success:
            print("❌ Failed to grab frame")
            break

        # Reset vehicle counter
        vehicle_count = 0

        # Draw ROI
        cv2.polylines(frame, [ROI_POINTS], isClosed=True, color=(255, 0, 0), thickness=2)

        # Run YOLO detection
        results = model(frame)

        for box in results[0].boxes:
            cls_id = int(box.cls)
            conf = float(box.conf[0])

            if cls_id in VEHICLE_CLASS_IDS and conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

                if cv2.pointPolygonTest(ROI_POINTS, (center_x, center_y), False) >= 0:
                    vehicle_count += 1
                    label = f"{model.names[cls_id]} {conf:.2f}"

                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Display count
        cv2.putText(frame, f"Vehicles in ROI: {vehicle_count}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Show live feed
        cv2.imshow("YOLO Vehicle Detection - Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    print("🛑 Releasing resources...")
    cap.release()
    cv2.destroyAllWindows()
